In [1]:
# Imports and definitions
import os
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, Activation
from keras import optimizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint 
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True  #To address OSError: image file is truncated 

# Testing Xception implementation.
from keras.applications import xception

# Paths to images 
training_path = 'dogImages/train'
validation_path = 'dogImages/valid'
test_path = 'dogImages/test'

Using TensorFlow backend.


I'm going to use transfer learning for this. The following steps create the bottleneck features for the Xception CNN with data augmentation. 

generate_bottelneck_features_n_labels(generator, batch_size, model): Takes in the data generator and model, and it returns the bottleneck_features and the corresponding labels.

In [2]:
# Since Keras doesn't seem to give an option to get labels with predict_generator, doing it manually. 
def generate_bottelneck_features_n_labels(generator, batch_size, model):
    ind = 0
    list_batches = []
    list_labels = []
    for images, label in generator:
        bott_features = model.predict(images)
        list_batches.append(bott_features)
        list_labels.append(label)
        if generator.samples//batch_size+1 <= ind:
            break
        ind += 1
    bottleneck_features = np.vstack(list_batches)
    labels = np.vstack(list_labels)
    return bottleneck_features, labels

In [3]:
if (not os.path.isfile('bottleneck_features/bf_data_aug_xception_train.npz')) \
    or (not os.path.isfile('bottleneck_features/bf_data_aug_xception_validation.npz')) \
    or (not os.path.isfile('bottleneck_features/bf_data_aug_xception_test.npz')):
    
    
    batch_size = 32
    # Instanciate the Xception model. Don't include the FC layers.
    xception_model = xception.Xception(include_top=False, weights='imagenet')

    # Data augmentation on the training data.
    data_generator_train = ImageDataGenerator(
                                        rescale=1./255,
                                        zoom_range=0.2,
                                        rotation_range=10,
                                        width_shift_range=0.1,
                                        height_shift_range=0.1,
                                        horizontal_flip=True)

    # data generator for validation & test data.
    data_generator_test = ImageDataGenerator(rescale=1./255)

    train_generator = data_generator_train.flow_from_directory(training_path, 
                                                        target_size=(224, 224), 
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        shuffle=False)

    validation_generator = data_generator_test.flow_from_directory(validation_path, 
                                                        target_size=(224, 224), 
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        shuffle=False)

    test_generator = data_generator_test.flow_from_directory(test_path, 
                                                        target_size=(224, 224), 
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        shuffle=False)
    
    
    train_xception, labels_train_exception = generate_bottelneck_features_n_labels(train_generator, batch_size, xception_model)
    valid_xception, labels_valid_exception = generate_bottelneck_features_n_labels(validation_generator, batch_size, xception_model)
    test_xception, labels_test_exception = generate_bottelneck_features_n_labels(test_generator, batch_size, xception_model)
    
    np.savez(open('bottleneck_features/bf_data_aug_xception_train.npz', 'wb'), train_xception=train_xception, labels_train_exception=labels_train_exception)
    np.savez(open('bottleneck_features/bf_data_aug_xception_validation.npz', 'wb'), valid_xception=valid_xception, labels_valid_exception=labels_valid_exception)
    np.savez(open('bottleneck_features/bf_data_aug_xception_test.npz', 'wb'), test_xception=test_xception, labels_test_exception=labels_test_exception)
    
else:
    train_data_xception = np.load('bottleneck_features/bf_data_aug_xception_train.npz')
    train_xception = train_data_xception['train_xception']
    labels_train_exception = train_data_xception['labels_train_exception']
    
    valid_data_xception = np.load('bottleneck_features/bf_data_aug_xception_validation.npz')
    valid_xception = valid_data_xception['valid_xception']
    labels_valid_exception = valid_data_xception['labels_valid_exception']
    
    test_data_xception = np.load('bottleneck_features/bf_data_aug_xception_test.npz')
    test_xception = test_data_xception['test_xception']
    labels_test_exception = test_data_xception['labels_test_exception']

Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.
Found 836 images belonging to 133 classes.


Now that I have the bottleneck features, create the NN for the FC layers.

In [4]:
my_model = Sequential()
my_model.add(GlobalAveragePooling2D(input_shape=train_xception.shape[1:]))
# my_model.add(Dense(1000, activation='relu'))
# my_model.add(BatchNormalization())
# my_model.add(Dropout(0.2))
# my_model.add(Dense(500, activation='relu'))
# my_model.add(BatchNormalization())
my_model.add(Dropout(0.3))
my_model.add(Dense(133, activation='softmax'))
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 133)               272517    
Total params: 272,517.0
Trainable params: 272,517.0
Non-trainable params: 0.0
_________________________________________________________________


Define optimizer, loss function and metric.

In [5]:
opt = optimizers.Adam(lr=0.002)
my_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Train the model.

In [6]:
epochs = 10
batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.xception_transfer.hdf5', 
                               verbose=1, save_best_only=True)

my_model.fit(train_xception, labels_train_exception, 
          validation_data=(valid_xception, labels_valid_exception),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Train on 6712 samples, validate on 867 samples
Epoch 1/10
6712/6712 [==============================] - 3s - loss: 1.2620 - acc: 0.6742 - val_loss: 0.5636 - val_acc: 0.8235
Epoch 2/10
6712/6712 [==============================] - 2s - loss: 0.4459 - acc: 0.8580 - val_loss: 0.5256 - val_acc: 0.8316
Epoch 3/10
6712/6712 [==============================] - 2s - loss: 0.2857 - acc: 0.9115 - val_loss: 0.4989 - val_acc: 0.8362
Epoch 4/10
6712/6712 [==============================] - 2s - loss: 0.2089 - acc: 0.9349 - val_loss: 0.5163 - val_acc: 0.8362
Epoch 5/10
6712/6712 [==============================] - 2s - loss: 0.1513 - acc: 0.9569 - val_loss: 0.4847 - val_acc: 0.8501
Epoch 6/10
6712/6712 [==============================] - 2s - loss: 0.1198 - acc: 0.9669 - val_loss: 0.4924 - val_acc: 0.8431
Epoch 7/10
6712/6712 [==============================] - 3s - loss: 0.0925 - acc: 0.9768 - val_loss: 0.4755 - val_acc: 0.8420
Epoch 8/10
6712/6712 [==============================] - 2s - loss: 0.0796 - ac

Load the best weights for the model.

In [7]:
my_model.load_weights('saved_models/weights.best.xception_transfer.hdf5')

Accuracy over test data set.

In [8]:
# Expanding dimensions for the input features, adding 1 for the num of samples. 
# So it matches with the expected input from Keras
# Using argmax to retrieve the index of the dog breed with maximum probability.
Xception_predictions = [np.argmax(my_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_xception]

# report test accuracy
# Checking indexes from predictions and test labels.
test_accuracy = 100*np.sum(np.array(Xception_predictions)==np.argmax(labels_test_exception, axis=1))/len(Xception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 83.8710%


83% Accuracy with transfer learning but no fine tunning. 
Fine tunning:

In [20]:
from keras.models import Model

xception_model = xception.Xception(include_top=False, weights='imagenet')
# Taking the output on the Conv layers.
bottleneck_output = xception_model.output
fc_layers = GlobalAveragePooling2D()(bottleneck_output)
fc_layers = Dropout(0.3)(fc_layers)
y_hat = Dense(133, activation='softmax')(fc_layers)
transfer_model = Model(inputs=xception_model.input, outputs=y_hat)

# Finetunning the last two big convolutional blocks Blocks12/13
for layer in transfer_model.layers[:-19]:
    layer.trainable = False

transfer_model.summary()    

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_15 (InputLayer)            (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864                                          
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128                                          
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0                                            
___________________________________________________________________________________________

In [21]:
opt = optimizers.Adam(lr=0.002)
transfer_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [23]:
batch_size = 32
epochs = 10
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.xception_transfer.finetunning.hdf5', 
                               verbose=1, save_best_only=True)

# Data augmentation on the training data.
data_generator_train = ImageDataGenerator(
                                    rescale=1./255,
                                    zoom_range=0.2,
                                    rotation_range=10,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    horizontal_flip=True)

# data generator for validation & test data.
data_generator_test = ImageDataGenerator(rescale=1./255)

train_generator = data_generator_train.flow_from_directory(training_path, 
                                                    target_size=(224, 224), 
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

validation_generator = data_generator_test.flow_from_directory(validation_path, 
                                                    target_size=(224, 224), 
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)

transfer_model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size, 
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples//batch_size,
                    epochs=epochs, callbacks=[checkpointer], verbose=1)

Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.
Epoch 1/10
208/208 [==============================] - 149s - loss: 1.4207 - acc: 0.5980 - val_loss: 1.4122 - val_acc: 0.6490
Epoch 2/10
208/208 [==============================] - 118s - loss: 0.9751 - acc: 0.7051 - val_loss: 1.5195 - val_acc: 0.6451
Epoch 3/10
208/208 [==============================] - 119s - loss: 0.7732 - acc: 0.7587 - val_loss: 1.2452 - val_acc: 0.6961
Epoch 4/10
208/208 [==============================] - 118s - loss: 0.6316 - acc: 0.8028 - val_loss: 1.2992 - val_acc: 0.6743
Epoch 5/10
208/208 [==============================] - 118s - loss: 0.5549 - acc: 0.8263 - val_loss: 1.2899 - val_acc: 0.7024
Epoch 6/10
208/208 [==============================] - 118s - loss: 0.4845 - acc: 0.8470 - val_loss: 1.2856 - val_acc: 0.7011
Epoch 7/10
208/208 [==============================] - 118s - loss: 0.4265 - acc: 0.8618 - val_loss: 1.2550 - val_acc: 0.7139
Epoch 8/10
208/208 [==================

In [27]:
my_model.load_weights('saved_models/weights.best.xception_transfer.finetunning.hdf5')
# Expanding dimensions for the input features, adding 1 for the num of samples. 
# So it matches with the expected input from Keras
# Using argmax to retrieve the index of the dog breed with maximum probability.


Xception_finetune_predictions = [np.argmax(transfer_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_generator]

# report test accuracy
# Checking indexes from predictions and test labels.
test_accuracy = 100*np.sum(np.array(Xception_finetune_predictions)==np.argmax(labels_test_exception, axis=1))/len(Xception_finetune_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

NameError: name 'test_generator' is not defined